In [1]:
# imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from IPython.display import Markdown, display, update_display
import fitz  # PyMuPDF

In [2]:
# Set up the environment 
load_dotenv(override = True)
api_key = os.getenv('OPENAI_API_KEY')

# Define the model 
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:

# Headers to mimic a real browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class JobPage:
    """
    Extracts job description from a company job listing page.
    """
    def __init__(self, url):
        self.url = url
        self.job_description = ""
        self._fetch_description()

    def _fetch_description(self):
        response = requests.get(self.url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Try known class names / tags commonly used for job descriptions
        possible_selectors = [
            {"name": "div", "class_": "job-description"},
            {"name": "section", "class_": "job-description"},
            {"name": "div", "class_": "content"},
            {"name": "div", "class_": "ats-description"},
            {"name": "div", "id": "job-details"},
            {"name": "div", "class_": "job-details"},
        ]

        for selector in possible_selectors:
            desc = soup.find(selector.get("name"), class_=selector.get("class_"), id=selector.get("id"))
            if desc:
                self.job_description = desc.get_text(separator="\n", strip=True)
                break

        # Fallback: remove irrelevant tags and extract main body text
        if not self.job_description and soup.body:
            for tag in soup.body(["script", "style", "img", "input"]):
                tag.decompose()
            self.job_description = soup.body.get_text(separator="\n", strip=True)

    def get_job_description(self):
        return self.job_description


# Example usage:
link = 'https://careers.telus.com/job/Terrace-Manager-Community-&-Service-Excellence-Terrace-BC/590496617/'
#job = JobPage(link)
#print("\nJob Description:\n")
#print(job.get_job_description())


In [4]:
def load_resume_pdf(path):
    doc = fitz.open(path)
    return "\n".join([page.get_text() for page in doc])

resume_text = load_resume_pdf("Mat_Imaginary_Resume.pdf")


In [5]:
# Define a system prompt 
system_prompt_cover_letter = (
    "You are a professional career assistant. You will be given a job description and a candidate's resume. "
    "Your task is to write a personalized cover letter explaining why the candidate is a great fit for the position. "
    "Use a tone that is professional, confident, and tailored to the job role. "
    "Base the letter only on information provided in the resume and job description. "
    "Highlight the most relevant experiences, skills, and accomplishments that align with the job. "
    "Avoid generic statements; be specific and targeted."
    "Respond in markdown"
)


# Define a user-prompt 
def user_prompt_cover_letter(url, resume_text):
    job_description = JobPage(url).get_job_description()
    prompt = (
        f"Job Description:\n{job_description}\n\n"
        f"Candidate Resume:\n{resume_text}\n\n"
        "Please write a customized cover letter explaining why this candidate is a great fit for the role, "
        "based on the job description and their professional experience. "
        "The letter should be concise (3-5 paragraphs), engaging, and demonstrate alignment with the job requirements."
    )
    return prompt


In [6]:
def message_cover_letter(url,resume_text): 
    return [
        {'role': 'system', 'content': system_prompt_cover_letter}, 
        {'role': 'user', 'content': user_prompt_cover_letter(url,resume_text)[:20_000]}
    ] 

# Example usage: 
#link = 'https://careers.telus.com/job/Terrace-Manager-Community-&-Service-Excellence-Terrace-BC/590496617/'
#message_cover_letter(link, resume_text)

In [7]:
def create_cover_letter(url, resume_text): 
    response = openai.chat.completions.create(
        model = MODEL, 
        messages = message_cover_letter(url,resume_text)
    )
    return response.choices[0].message.content

In [8]:
def stream_cover_letter(url, resume_text):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages= message_cover_letter(url,resume_text), 
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [9]:
link = 'https://careers.telus.com/job/Terrace-Manager-Community-&-Service-Excellence-Terrace-BC/590496617/'
stream_cover_letter(link,resume_text)


[Your Name]  
[Your Address]  
[Your City, State, Zip]  
[Your Email]  
[Your Phone Number]  
[Date]  

Hiring Manager  
TELUS  
Terrace, BC, CA  

Dear Hiring Manager,

I am excited to submit my application for the Manager - Community & Service Excellence position at TELUS, as I believe my background in product management and team leadership aligns closely with the requirements outlined in your job description. With five years of experience at IBM, where I successfully managed cross-functional teams and drove product strategy, I am confident in my ability to lead and mentor your team while ensuring exceptional service delivery within the community.

In my current role as Product Manager & Team Lead, I have demonstrated strong leadership by guiding a team of 12 engineers and designers to develop AI-powered applications. My experience leading agile transformations has not only increased delivery efficiency by 30% but has also instilled a culture of continuous improvement among my colleagues. I am adept at inspiring my team to excel, fostering talent development and driving operational rigor—key elements that are essential for the role you seek to fill. 

Additionally, my ability to build strategic relationships with stakeholders has been pivotal in aligning project objectives with customer needs. I have a proven track record of conducting comprehensive competitive analyses and user research, leading to enhancements that drive customer satisfaction and engagement. This aligns perfectly with TELUS’s commitment to community ownership and the expectation to manage operational demands effectively while emphasizing safety and performance.

I am particularly drawn to TELUS’s vision of integrating community engagement with business success. My desire to drive outcomes for the community and my passion for technology solutions position me as an ideal candidate who can represent TELUS’s values and elevate the brand. I am eager to contribute to a high-performance culture that inspires both team members and customers alike.

Thank you for considering my application. I look forward to discussing how my experience and vision can contribute to TELUS's mission of delivering unparalleled customer service excellence.

Warm regards,

Mat Imaginary  
mat.imaginary@example.com  
(555) 123-4567  

